<a href="https://colab.research.google.com/github/shouvickp/image-search/blob/main/FAISS_VECTOR_SEARCH_DATABASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Colab!

In [1]:
!pip install -q git+https://github.com/openai/CLIP.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.6 MB/s eta 0:00:00


In [2]:
!pip install -q ftfy regex tqdm

In [3]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.2 MB/s eta 0:00:00


In [4]:
import torch
import clip
import faiss
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm


In [5]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

Mounted at /content/drive


In [6]:
df = pd.read_csv('/content/drive/My Drive/4th sem questions/fashion_sample_5000.csv')
print(df.head())

   id                                    name  \
0   1              Men Solid Oversized Cotton   
1   2          Men Cotton Pure Cotton T-shirt   
2   3               Women Pure Cotton T-shirt   
3   4                Typography Print T-shirt   
4   5  Printed Round Neck Pure Cotton T-shirt   

                                                 img  price     mrp  rating  \
0  https://assets.myntassets.com/f_webp,dpr_1.0,q...  532.0  1299.0     4.1   
1  https://assets.myntassets.com/f_webp,dpr_1.0,q...  274.0   499.0     4.2   
2  https://assets.myntassets.com/f_webp,dpr_1.0,q...  551.0  1199.0     4.5   
3  https://assets.myntassets.com/f_webp,dpr_1.0,q...  296.0  1099.0     4.0   
4  https://assets.myntassets.com/f_webp,dpr_1.0,q...  494.0   899.0     4.2   

   ratingTotal  discount  
0         5300        59  
1        25400        45  
2         3400        54  
3        18400        73  
4         3800        45  


In [7]:
import torch
import clip
import faiss
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess = clip.load("ViT-B/32", device=device)


100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 74.9MiB/s]


In [9]:
def get_image_embedding(image_url):
    try:
        response = requests.get(image_url, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image = preprocess(image).unsqueeze(0).to(device)

        with torch.no_grad():
            embedding = model.encode_image(image)

        return embedding.cpu().numpy()[0]
    except:
        return None


In [10]:
df_5 = df.head(5).copy()

In [11]:
embeddings = []

df_5 = df.head(5).copy()

for url in tqdm(df_5["img"]):
    emb = get_image_embedding(url)
    embeddings.append(emb)

df_5["embedding"] = embeddings
df_5 = df_5[df_5["embedding"].notnull()].reset_index(drop=True)


100%|██████████| 5/5 [00:01<00:00,  2.95it/s]


In [12]:
embedding_dim = len(df_5["embedding"][0])

index = faiss.IndexFlatL2(embedding_dim)
index.add(np.vstack(df_5["embedding"].values))

print("Total vectors:", index.ntotal)


Total vectors: 5


In [13]:
def recommend_products(image_url, top_k=1):
    uploaded_image_path = requests.get(image_url, timeout=10)
    image = Image.open(BytesIO(uploaded_image_path.content)).convert("RGB")
    image = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        query_embedding = model.encode_image(image).cpu().numpy()

    distances, indices = index.search(query_embedding, top_k)

    return df_5.iloc[indices[0]][[
        "name", "price", "rating", "discount", "img"
    ]]


In [14]:
recommend_products("https://assets.myntassets.com/f_webp,dpr_1.0,q_60,w_210,c_limit,fl_progressive/assets/images/11545192/2022/3/25/626b0419-cbb0-4200-8490-67e20a0021c51648208612203-Roadster-Men-Green--White-Printed-Round-Neck-Pure-Cotton-T-s-1.jpg")

,name,price,rating,discount,img
4,Printed Round Neck Pure Cotton T-shirt,494.0,4.2,45,"https://assets.myntassets.com/f_webp,dpr_1.0,q..."


In [15]:
recommend_products("https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/29861614/2024/5/30/e682304c-1f49-4564-ae74-485b23f419001717082617553MonteCarloMenPrintedPocketsT-shirt1.jpg")

,name,price,rating,discount,img
4,Printed Round Neck Pure Cotton T-shirt,494.0,4.2,45,"https://assets.myntassets.com/f_webp,dpr_1.0,q..."


In [16]:
recommend_products("https://img01.ztat.net/article/spp-media-p1/e555cec632fa4b5680a741655e219972/e14c84c246ea48a896d1eb1187240b86.jpg")

,name,price,rating,discount,img
2,Women Pure Cotton T-shirt,551.0,4.5,54,"https://assets.myntassets.com/f_webp,dpr_1.0,q..."
